In [ ]:
import matplotlib.pyplot as plt

import json
import numpy as np
import fitsio as ft
import healpy as hp
import pandas as pd
import sys

sys.path.insert(0, '/home/mehdi/github/LSSutils')
from LSSutils.catalogs.combinefits import EbossCatalog
from LSSutils.lab import get_cl, MeanDensity, make_overdensity
%matplotlib inline

In [ ]:
from scipy.stats import pearsonr

In [ ]:
from LSSutils.utils import EbossCat

In [ ]:
# read the NN N_models
nmodel_0 = hp.read_map('../output/eboss_ngal_ngc_all_pnll_kfold.hp512.fits', verbose=False)
nmodel_1 = hp.read_map('../output/eboss_ngal_ngc_all_pnll_kfold_seed85.hp512.fits', verbose=False)
nmodel_2 = hp.read_map('../output/eboss_ngal_nranexp_ngc_all_pnll_kfold.hp512.fits', verbose=False)
nmodel_3 = hp.read_map('../output/eboss_ngal_ngc_all_pnll_kfold_seed1to6.hp512.fits', verbose=False)
nmodel_4 = hp.read_map('../output/eboss_ngal_ngc_all_pnll_kfold_seed42.hp512.fits', verbose=False)
nmodel_5 = hp.read_map('../output/eboss_ngal_ngc_all_pnll_kfold_seed2020.hp512.fits', verbose=False)
nmodel_6 = hp.read_map('../output/eboss_ngal_ngc_all_plain_pnll_kfold_seed42.hp512.fits', verbose=False)

In [ ]:
good = np.isfinite(nmodel_0)
nmodel_0[good], nmodel_4[good]

In [ ]:
# test changing fpix and seed
from scipy.stats import pearsonr, spearmanr

plt.scatter(nmodel_0[good], nmodel_1[good], 1, label='different seed, same fpix', alpha=0.5)
plt.scatter(nmodel_0[good], nmodel_2[good], 1, label='same seed, different fpix', alpha=0.5)
plt.scatter(nmodel_0[good], nmodel_3[good], 1, label='different seed, same fpix', alpha=0.5, color='r')
plt.scatter(nmodel_0[good], nmodel_5[good], 1, label='different seed, same fpix', alpha=0.5)

plt.plot([0.2, 1.5], [0.2, 1.5], ls='--', color='b', label='y=x')
plt.plot([0.2, 1.5], [0.2, 1.58*1.5], ls='--', color='orange', label='y=1.58x')

plt.annotate(f'{pearsonr(nmodel_0[good], nmodel_2[good])[0]:.2f}', 
             (0.9, 0.82), xycoords='axes fraction', color='orange')
plt.annotate(f'{pearsonr(nmodel_0[good], nmodel_1[good])[0]:.2f}', 
             (0.9, 0.62), xycoords='axes fraction', color='blue')

plt.legend(markerscale=5)
plt.xlabel('Nmodel1')
plt.ylabel('Nmodel2')

In [ ]:
# test changing fpix and seed

# plt.scatter(nmodel_1[good], nmodel_2[good], 1, label='different seed, same fpix', alpha=0.5)
plt.scatter(nmodel_3[good], nmodel_1[good], 1, label='same seed, different fpix', alpha=0.5)
plt.scatter(nmodel_3[good], nmodel_4[good], 1, label='different seed, same fpix', alpha=0.5)
plt.scatter(nmodel_3[good], nmodel_5[good], 1, label='different seed, same fpix', alpha=0.5)

plt.plot([0.2, 1.5], [0.2, 1.5], ls='--', color='b', label='y=x')
# plt.plot([0.2, 1.5], [0.2, 1.58*1.5], ls='--', color='orange', label='y=1.58x')

plt.legend(markerscale=5)
plt.xlabel('Nmodel1')
plt.ylabel('Nmodel2')


In [ ]:
# print the correlation coef.
nmodels = [nmodel_0, nmodel_1, nmodel_2, nmodel_3, nmodel_4, nmodel_5]
for i in range(6):
    for j in range(i+1, 6):
        print(i, j, pearsonr(nmodels[i][good], nmodels[j][good])[0])
        
for i in range(6):
    print(i, np.std(nmodels[i][good]))

In [ ]:
# test angular clustering

# read data and randoms
data = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.dat.fits', kind='galaxy', zmin=0.8, zmax=2.2)
randoms = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.ran.fits', kind='random', zmin=0.8, zmax=2.2)

# to HEALPix
nobs_raw = data.tohp(512, 0.8, 2.2, raw=0)
nobs_w = data.tohp(512, 0.8, 2.2, raw=1)
nran_w = randoms.tohp(512, 0.8, 2.2, raw=1)

# fpix map
fpix = np.zeros_like(nran_w)
nran_mean = nran_w[nran_w>0].mean()
# nran_mean = 5000.*hp.nside2pixarea(512, degrees=True)
fpix[nran_w>0] = nran_w[nran_w>0] / nran_mean
print(f'nran_mean: {nran_mean}')

# weight map
weight = np.zeros_like(nobs_raw)
weight[nobs_raw>0] = nobs_w[nobs_raw>0]/nobs_raw[nobs_raw>0]
weight[nobs_raw==0] = 1.0 

In [ ]:
def normalize(nnmodel, nobsw, good):
    # normalize systematic weights such that N_{qso,tot} stays the same 
    norm = (nobsw[good]/nnmodel[good]).sum()/nobsw[good].sum()
    return norm*nnmodel

nmodel_0s = normalize(nmodel_0, nobs_w, good)
nmodel_1s = normalize(nmodel_1, nobs_w, good)
nmodel_2s = normalize(nmodel_2, nobs_w, good)
nmodel_3s = normalize(nmodel_3, nobs_w, good)
nmodel_4s = normalize(nmodel_4, nobs_w, good)
nmodel_5s = normalize(nmodel_5, nobs_w, good)
nmodel_6s = normalize(nmodel_6, nobs_w, good)

In [ ]:
print('N_{qso,tot}: %.1f %.1f'%(nobs_w[good].sum(), 
                               (nobs_w[good]/nmodel_0s[good]).sum()))

In [ ]:
_,b,_ = plt.hist(nobs_raw[good], bins=30, label=r'N$_{qso, {\rm raw}}$')
_=plt.hist(nmodel_0s[good], bins=b, alpha=0.8, label=r'$\lambda$')
plt.legend(fontsize=15)

In [ ]:
np.percentile(1/nmodel_0s[good], [0, 100])

In [ ]:
plt.hist(1/nmodel_0[good], alpha=0.8, bins=50,
        range=(0.5, 2.2), label=r'$N^{-1}_{\rm model}$')
_=plt.hist(1/nmodel_0s[good], alpha=0.8, bins=50, 
           range=(0.5, 2.2), label=r'$(\beta N_{\rm model})^{-1}$')
plt.legend()

In [ ]:
def plot_nobs_nmodel(nmodel, nobs, ax, xlabel, ylabel, 
                     xlim=(-0.5, 2), ylim=(-1, 8), add_line=True, color='r', label='Nmodel'):
    y0_, x_, _ = binned_statistic(nmodel, nmodel, statistic='mean', bins=6)
    y1_, x_, _ = binned_statistic(nmodel, nobs, statistic='mean', bins=x_)
    y2_, x_, _ = binned_statistic(nmodel, nobs, statistic=np.std, bins=x_)
    n2_, x_, _ = binned_statistic(nmodel, np.ones_like(nobs), statistic='sum', bins=x_)
    x_c = 0.5*(x_[1:]+x_[:-1])

    ax.scatter(nmodel, nobs, 1, marker='.', color=color, zorder=-1, alpha=0.1)
        
    ax.errorbar(y0_, y1_, yerr=y2_/np.sqrt(n2_), color=color, 
                ls='none', capsize=2, label=label, marker='o')
    ax.errorbar(y0_, y1_, yerr=y2_, color=color, 
                ls='none', capsize=2, alpha=0.5)

    #ax.set(xlabel=xlabel, ylabel=ylabel, xlim=xlim, ylim=ylim)
    return ax

In [ ]:
from scipy.stats import binned_statistic

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot([0.2, 1.5], [0.2, 1.5], ls='--', color='b', label='y=x')
ax.plot([0.2, 1.5], [0.2, 1.58*1.5], ls='--', color='orange', label='y=1.58x')

ax = plot_nobs_nmodel(nmodel_0[good], nmodel_1[good], ax, '', '',
                      color='b', label='different seed, same fpix')
ax = plot_nobs_nmodel(nmodel_0[good], nmodel_2[good], ax, 'modeli', 'modelj',
                      color='orange', label='same seed, different fpix')
ax = plot_nobs_nmodel(nmodel_0[good], nmodel_3[good], ax, '', '',
                      color='r', label='different seed, same fpix')

ax = plot_nobs_nmodel(nmodel_0[good], nmodel_5[good], ax, '', '',
                      color='g', label='different seed, same fpix')

ax.legend(title='X=')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot([0.2, 1.5], [0.2, 1.5], ls='--', color='b', label='y=x')
# ax.plot([0.2, 1.5], [0.2, 1.58*1.5], ls='--', color='orange', label='y=1.58x')

ax = plot_nobs_nmodel(nmodel_1[good], nmodel_3[good], ax, '', '',
                      color='b', label='different seed, same fpix')
# ax = plot_nobs_nmodel(nmodel_2[good], nmodel_0[good], ax, 'modeli', 'modelj',
#                       color='orange', label='same seed, different fpix')
# ax = plot_nobs_nmodel(nmodel_3[good], nmodel_0[good], ax, '', '',
#                       color='r', label='different seed, same fpix')
ax = plot_nobs_nmodel(nmodel_5[good], nmodel_1[good], ax, '', '',
                      color='g', label='different seed, same fpix')

ax.legend()

In [ ]:
nnchains = ft.read('/home/mehdi/data/eboss/data/v7_2/0.7/pnll_nranexp/nn-weights-fold0.fits')

In [ ]:
nnchains

In [ ]:
set1 = nnchains['weight'][:, :10]
set2 = nnchains['weight'][:, 10:]
set1mean  = np.mean(set1, 1)
set2mean  = np.mean(set2, 1)
set1median  = np.median(set1, 1)
set2median  = np.median(set2, 1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot([0.5, 1.9], [0.5, 1.9], ls='--', color='b', label='y=x')
# ax.plot([0.2, 1.5], [0.2, 1.58*1.5], ls='--', color='orange', label='y=1.58x')

ax = plot_nobs_nmodel(set1mean, set2mean, ax, '', '',
                      color='b', label='mean over 10 chains')

# ax = plot_nobs_nmodel(set1median, set2median, ax, '', '',
#                       color='r', label='median over 10 chains')

ax = plot_nobs_nmodel(set1[:, 0], set2[:, 1], ax, '', '',
                      color='orange', label='one chain')
ax.legend()

In [ ]:
pearsonr(set1[:, 0], set2[:, 1]), pearsonr(set1mean, set2mean)

In [ ]:
nmodel_3s = hp.read_map('/home/mehdi/data/eboss/data/v7_2/0.3/results/NGC_all_512/regression/nn_known/nn-weights.hp512.fits', verbose=False)

In [ ]:
templates = pd.read_hdf('../input/SDSS_WISE_HI_imageprop_nside512.h5', key='templates')

In [ ]:
systematics = templates[['star_density', 'ebv']].values
systematics.shape

In [ ]:
for i in range(2):
    print(good.sum())
    good &= np.isfinite(systematics[:, i])    

In [ ]:
from time import time

In [ ]:
cells = {}

for slf, ni in zip([None, nmodel_0s, nmodel_1s, nmodel_2s, nmodel_3s, nmodel_6s],
                    ['before', 'after0', 'after1', 'after2', 'default', 'after.plain']):
    t0 = time()
    cells[ni] = get_cl(nobs_w, nran_w, good, systematics=systematics, selection_fn=slf, njack=0)
    print('\n', ni, time()-t0)

In [ ]:
from LSSutils.utils import histogram_cell

In [ ]:
def add_cl(ax, cli, shade, **kw):
    for i in range(2):        
        lb, clb_sg = histogram(cli['cl_gg']['l'], cli['cl_sg'][i]['cl'])
        lb, clb_ss = histogram(cli['cl_gg']['l'], cli['cl_ss'][i]['cl'])
        
        clb = clb_sg*clb_sg / clb_ss
        
        ax[i].plot(lb, clb, marker='.', ms=10, **kw)
        if shade:
            lb, clb_e = histogram_jac(cli['cl_gg']['l'], cli['cl_gg'][i]['cl'])
            ax[i].fill_between(lb, 0, clb_e, color='grey', alpha=0.1, zorder=-1, label=r'$\sigma C_{g,g}$')

    return ax


fig, ax = plt.subplots(ncols=2, figsize=(12, 4), sharey=True)
fig.subplots_adjust(wspace=0)

add_cl(ax, cells['before'], True, color='grey', label='before')

colors = ['k', 'r', 'b', 'orange']
i = 0
for name in ['after0', 'after1', 'after2', 'default']:
    print(name)
    ax = add_cl(ax, cells[name], False, color=colors[i], label=name)
    i += 1

ax[1].legend()
for axi in ax:
    axi.set(xscale='log',  xlabel=r'$\ell$', 
            yscale='log', ylim=(1.0e-12, 1.0e-3)) #
    axi.grid(True, ls=':', alpha=0.4)
ax[0].set_ylabel(r'$C^{2}_{\ell,s,g}/C_{s,s}$', fontsize=15)# 
fig.savefig('cell_eboss.png', dpi=300, bbox_inches='tight')

In [ ]:
y_ = ft.read('/home/mehdi/data/eboss/data/v7_2/0.6/eboss_ngal_ngc_all_512.fits')
ebv = np.zeros_like(nobs_w)
ebv[y_['hpind']] = y_['features'][:, 2]

In [ ]:
nbars = {}

for slf, ni in zip([None, nmodel_0s, nmodel_1s, nmodel_2s, nmodel_3s, nmodel_6s],
                    ['before', 'after0', 'after1', 'after2', 'default', 'after.plain']):
    nbar_i = MeanDensity(nobs_w/fpix, np.ones_like(good), good, np.log10(ebv), selection=slf)
    nbar_i.run()
    nbars[ni] = nbar_i.output

In [ ]:
chi2 = lambda y, y_, ye:(((y-y_)/ye)**2).sum()

fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

mk = ['o', 's', 'x', '+', '*', '^']
cl = ['grey', 'k', 'r', 'b', 'orange', 'g']
i = 0
for ni, nli in nbars.items():
    
    ax[0].plot(nli['bin_avg'], nli['nnbar'], label=ni, marker=mk[i], color=cl[i])
    ax[0].text(0.1, 0.1+i*0.1, f"{ni}, {chi2(1, nli['nnbar'], nli['nnbar_err']):.1f}",
              transform=ax[0].transAxes, color=cl[i])
    i += 1
    
ax[0].set(xlabel=r'$\log$(EBV)', ylabel='N/Nbar')

# i = 0
# for ni, cli in cells.items():
    
#     lb, clb = histogram_cell(cli['cl_gg']['cl'])
#     ax[1].loglog(lb, clb, label=ni, marker=mk[i], color=cl[i])
#     i += 1 

# ax[1].legend()
# ax[1].set(ylim=(1.0e-6, 3.0e-4), ylabel=(r'C$_{\ell}$'), xlabel=r'$\ell$')
for axi in ax:
    axi.grid(True, ls=':', alpha=0.6)
fig.savefig('eboss_ngc_all_ratio.png', bbox_inches='tight', dpi=300)

In [ ]:
nnbar_ngc = np.load('/home/mehdi/data/eboss/data/v7_2/0.3/old/clustering/nnbar_0.3_NGC_known_allhigh_all.npy', allow_pickle=True)

In [ ]:
nnbar_ngc[2]

In [ ]:
for nni in [nbar_d, nnbar_ngc[2]

In [ ]:
for dli, ni in zip([delta_before, delta_default, delta_after],
                   ['before', 'default (MSE)', 'Poisson']):
    plt.hist(dli[good], range=(-2, 10), bins=50, label=ni, alpha=0.5)
    print(np.percentile(dli[good], [0, 100]), (dli[good]*fpix[good]).mean())
plt.legend()
plt.xlim(-1.1, 10.)
plt.yscale('log')
plt.xlabel(r'$\delta_{qso, X}$')

In [ ]:
plt.hisw_sys1[good]

In [ ]:
data['WEIGHT_SYSTOT'].mean()

In [ ]:
np.mean(w_sys1[good])

In [ ]:
lambda_nn[good].min()

In [ ]:
np.min(lambda_nn_['pred'])

In [ ]:
from LSSutils.lab import overdensity

In [ ]:
delta_nn = np.zeros_like(nn_model)
delta_nn[good] = (nobs_raw[good]/fpix[good]*weight[good])/nn_model[good] - 1

delta_nnb = np.zeros_like(nn_model)
delta_nnb[good] = (nobs_raw[good]/fpix[good]*weight[good])/(beta*nn_model[good]) - 1

delta_nnnb = np.zeros_like(nn_model)
delta_nnnb[good] = (nobs_raw[good]/fpix[good]*weight[good])/(nbar*nn_model[good]) - 1

In [ ]:
%matplotlib inline

In [ ]:
nn_model[good].mean(), nobs_w[good].mean(), nobs_raw[good].mean(), (nn_model[good]*beta).mean()

In [ ]:
good_w_mask = good & (nn_model>0.5) & (nn_model<2.0)
good.sum(), good_w_mask.sum()

In [ ]:
_,b,_=plt.hist(delta_nn[good], bins=30)
# plt.hist(delta_nn[good_w_mask], bins=b, label='0.5<Nmodel<2.0', alpha=0.5)
plt.yscale('log')
plt.xlabel(r'\delta')

In [ ]:
for di in [delta_nn, delta_nnb, delta_nnnb]:
    print(di[good].mean(), di[good_w_mask].mean())

$$
    Cost = \sum (Ngal_{\rm obs, i} - Nmodel_{i} * fpix_{i}/weight_{i})^{2}
$$

In [ ]:
import pandas as pd
import fitsio as ft
import numpy as np
import healpy as hp
from time import time

#import torch
#from torch import nn

from astropy.table import Table
import logging
import json

import sys
sys.path.insert(0, '/home/mehdi/github/LSSutils')
#sys.path.append('/home/mehdi/github/sysnetdev')
#from sysnet.sources import set_logger

from scipy.stats import binned_statistic, lognorm

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from LSSutils.catalogs.combinefits import EbossCatalog, HEALPixDataset
from LSSutils.catalogs.datarelease import cols_eboss_mocks_qso

In [ ]:
''' read nn outputs
'''
with open('../output/eboss_ngal_ngc_all_pnll_kfold.json', 'r') as file:
    lambda_nn_ = json.load(file)
    
lambda_nn = np.zeros(12*512*512)
lambda_nn[lambda_nn_['hpix']] = lambda_nn_['pred']
    
with open('../output/eboss_ngal_nranexp_ngc_all_pnll_kfold.json', 'r') as file:
    lambda_nnran_ = json.load(file)
lambda_nnran = np.zeros(12*512*512)
lambda_nnran[lambda_nnran_['hpix']] = lambda_nnran_['pred']

        
# with open('../output/eboss_ngal_ngc_all_mse_kfold.json', 'r') as file:
#     y_gau = json.load(file)    
# assert np.array_equal(y_poi['hpix'], y_gau['hpix'])

# np.array_equal(y_poi['hpix'], y_poi_nran['hpix'])

In [ ]:
hpind = np.array(lambda_nn_['hpix'])
pred = np.array(lambda_nn_['pred']).astype('float32')
hpind, pred

In [ ]:
np.zeros()

In [ ]:
import healpy as hp

In [ ]:
python app.py -ax 0 2 5 13 --model dnnp --loss pnll -i /home/mehdi/data/eboss/data/v7_2/0.6/eboss_ngal_nranexp_ngc_all_512.fits -o ../output/eboss_ngal_nranexp_ngc_all_pnll_kfold.pt -ne 300 -k

In [ ]:
def tohp(nside, hpind, values):
    zeros = np.empty(12*nside*nside, dtype=values.dtype)*np.nan
    zeros[hpind] = values
    return zeros

In [ ]:
hppred = tohp(512, hpind, pred)

In [ ]:
hppred[np.isfinite(hppred)]

In [ ]:
# ngcall = ft.read('/home/mehdi/data/eboss/data/v7_2/0.6/eboss_ngal_ngc_all_512.fits')
# ngcall_nranexp = ft.read('/home/mehdi/data/eboss/data/v7_2/0.7/eboss_ngal_nranexp_ngc_all_512.fits')

In [ ]:
data = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.dat.fits', kind='galaxy', zmin=0.8, zmax=2.2)
randoms = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.ran.fits', kind='random', zmin=0.8, zmax=2.2)

In [ ]:
nobs_raw = data.tohp(512, 0.8, 2.2, raw=0)
nobs_w = data.tohp(512, 0.8, 2.2, raw=1)
nran_w = randoms.tohp(512, 0.8, 2.2, raw=1)

df = np.load('/home/mehdi/data/eboss/data/v7_2/0.3/ngal_features_NGC_all_512.5r.npy', allow_pickle=True).item()
df_all = np.concatenate([df['test']['fold%d'%d] for d in range(5)])

mask_default = np.zeros_like(nobs_w, '?')
mask_default[df_all['hpind']] = True
frac_raw = nran_w / (nran_w[nran_w>0]).mean()

nbar = nobs_w[mask_default].sum() / frac_raw[mask_default].sum()
wsys_default = hp.read_map('/home/mehdi/data/eboss/data/v7_2/0.3/results/NGC_all_512/regression/nn_known/nn-weights.hp512.fits')

In [ ]:
weight = nobs_w / nobs_raw
weight[nobs_raw==0] = 1.0

In [ ]:
nmodel_default_prime = nbar * frac_raw * wsys_default / weight # t ~ F^ Nobs x weight / (fpixNbar) ~ F^
nmodel_poisson_prime = lambda_nn

In [ ]:
def plot_nobs_nmodel(nobs, nmodel, ax, xlabel, ylabel, 
                     xlim=(-0.5, 2), ylim=(-1, 8), add_line=True, color='r', label='Nmodel'):
    y0_, x_, _ = binned_statistic(nmodel, nmodel, statistic='mean')
    y1_, x_, _ = binned_statistic(nmodel, nobs, statistic='mean', bins=x_)
    y2_, x_, _ = binned_statistic(nmodel, nobs, statistic=np.std, bins=x_)
    n2_, x_, _ = binned_statistic(nmodel, np.ones_like(nobs), statistic='sum', bins=x_)
    x_c = 0.5*(x_[1:]+x_[:-1])


    if add_line:        
        ax.plot([-.5, 2.], [-0.5, 2.], 'k-')
    ax.scatter(nmodel, nobs, 1, marker='.', color=color, zorder=-1)
        
    ax.errorbar(y0_, y1_, yerr=y2_/np.sqrt(n2_), color=color, 
                ls='none', capsize=2, label=label, marker='o')
    ax.errorbar(y0_, y1_, yerr=y2_, color=color, 
                ls='none', capsize=2, alpha=0.5)

    ax.set(xlabel=xlabel, ylabel=ylabel, xlim=xlim, ylim=ylim)
    return ax

In [ ]:
mask = lambda_nn_['hpix']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax = plot_nobs_nmodel(nobs_raw[mask], nmodel_poisson_prime[mask], ax, 'X', 'Nqso_obs',
                      label='lambda', color='orange')
ax = plot_nobs_nmodel(nobs_raw[mask], nmodel_default_prime[mask], ax, 'X', 'Nqso_obs',
                      color='b', label='Nbar*(fpix/weight)*F^', add_line=True)
ax.legend(title='X=')

In [ ]:
weight[mask]

In [ ]:
plt.scatter(lambda_nn[mask]*weight[mask], lambda_nnran[mask]*weight[mask], 1)
plt.plot([-0.1, 2.], [-0.1, 2.], 'k--')
plt.xlabel("weight*lambda [fpix w/ nranbar=41/sq deg]")
plt.ylabel("weight*lambda' [fpix w/ nranbar=65/sq deg]")

In [ ]:
def make_datasets():
    
    templates = pd.read_hdf('../input/SDSS_WISE_HI_imageprop_nside512.h5', key='templates')  
    data = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.dat.fits', kind='galaxy', zmin=0.8, zmax=2.2)
    randoms = EbossCatalog('../input/eBOSS_QSO_full_NGC_v7_2.ran.fits', kind='random', zmin=0.8, zmax=2.2)

    for coli in cols_eboss_mocks_qso:
        print(coli, end=', ')
        
    dataset = HEALPixDataset(data, randoms, templates, cols_eboss_mocks_qso)
    # nnbarall = dataset.prepare(512, 0.8, 2.2, label='nnbar')
    ngcall = dataset.prepare(512, 0.8, 2.2, label='ngal')
    ngcall_nranexp = dataset.prepare(512, 0.8, 2.2, label='ngal', nran_exp=65.)
    # only run once to save the data for regression
    # ft.write('/home/mehdi/data/eboss/data/v7_2/0.6/eboss_ngal_ngc_all_512.fits', ngcall, clobber=True)
    # ft.write('/home/mehdi/data/eboss/data/v7_2/0.7/eboss_ngal_nranexp_ngc_all_512.fits', ngcall_nranexp, clobber=True)
    return 'done'

def read_default():
    #n_obs_raw = data.tohp(512, 0.8, 2.2, raw=1)
    #n_ran_raw = randoms.tohp(512, 0.8, 2.2, raw=1)

    df = np.load('/home/mehdi/data/eboss/data/v7_2/0.3/ngal_features_NGC_all_512.5r.npy', allow_pickle=True).item()
    print(df.keys())

    df_all = np.concatenate([df['test']['fold%d'%d] for d in range(5)])
    #frac_raw = n_ran_raw / (n_ran_raw[n_ran_raw>0]).mean()
    print(np.array_equal(frac_raw[df_all['hpind']], df_all['fracgood']))

    delta_raw_default = np.zeros(12*512*512)

    mask_default = np.zeros_like(delta_raw_default, '?')
    mask_default[df_all['hpind']] = True

    sf = n_obs_raw[mask_default].sum() / frac_raw[mask_default].sum()
    delta_raw_default[mask_default] = n_obs_raw[mask_default]/(frac_raw[mask_default]*sf)

    print(np.array_equal(delta_raw_default[df_all['hpind']], df_all['label']))
    wsys_default = hp.read_map('/home/mehdi/data/eboss/data/v7_2/0.3/results/NGC_all_512/regression/nn_known/nn-weights.hp512.fits')    

Read

In [ ]:
nran = randoms.tohp(512, 0.8, 2.2, raw=1)
print(nran[nran>0].mean())
frac1 = nran / 65.
frac2 = nran / nran[nran>0].mean()
frac1 = frac1[y_poi['hpix']]
frac2 = frac2[y_poi_nran['hpix']]

In [ ]:
plt.scatter(y_poi['pred'], y_poi_nran['pred'], 1., alpha=0.5)

plt.xlabel("lambda")
plt.ylabel("lambda'")
plt.xlim(-0.1, 3.5)
plt.ylim(-0.1, 2.4)

In [ ]:
# organize w_sys as HEALPix
w_syspoi = np.zeros(12*512*512)
w_syspoi[y_poi['hpix']] = y_poi['pred'] 

#w_sysgau = np.zeros_like(w_syspoi)
#w_sysgau[y_gau['hpix']] = y_gau['pred'] 

# organize observed ngal, frac, mask, and ebv
y_ = ft.read('/home/mehdi/data/eboss/data/v7_2/0.6/eboss_ngal_ngc_all_512.fits')

n_obs = np.zeros(12*512*512)
n_obs[y_['hpind']] = y_['label']

fpix = np.zeros_like(n_obs)
fpix[y_['hpind']] = y_['fracgood']

mask = np.zeros_like(n_obs, '?')
mask[y_['hpind']] = True

ebv = np.zeros_like(n_obs)
ebv[y_['hpind']] = y_['features'][:, 2]

# np.setdiff1d(y_poi['hpix'], np.argwhere(mask).flatten()) []
# np.array_equal(ngcall['label'], y_['label']) True

In [ ]:
# construct the overdensity
delta_i = n_obs / fpix - 1 # fpix = fpix / weight
# delta_fg = n_obs / (fpix*w_sysgau) - 1
delta_fp = n_obs / (w_syspoi) - 1

mask_c = (w_syspoi/fpix>0.5) & (w_syspoi/fpix < 2.0)
mask_t = mask_c & mask
print(mask_c.sum(), mask.sum(), mask_t.sum())

In [ ]:
sf = (w_syspoi[mask_t]*fpix[mask_t]).sum()/n_obs[mask_t].sum()

In [ ]:
delta_fpp = (n_obs / (fpix*w_syspoi/sf))-1.

In [ ]:
delta_i[mask].mean(), delta_i[mask_t].mean(), delta_fpp[mask_t].mean(), delta_fp[mask].mean()

In [ ]:
delta_fg = n_obs / (fpix*wsys_default) - 1

In [ ]:
_=plt.hist(frac_raw, bins=50, range=(1.e-6, 2), )
plt.xlabel('fpix')

In [ ]:
nmodel_default_prime[mask].max()

In [ ]:
nmodel_default_prime2[mask].max()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax = plot_nobs_nmodel(nmodel_poisson_prime[mask], n_obs[mask], ax, 'Nobs', 'fpix x Nmodel / weight',  label='Poisson Nmodel', xlim=(-1, 8), ylim=(-1, 4))
ax = plot_nobs_nmodel(nmodel_default_prime[mask], n_obs[mask], ax, 'Nobs', 'fpix x Nmodel / weight',  color='b', add_scatter=False, label='Nbar x F^', xlim=(-1, 8), ylim=(-1, 4))
ax = plot_nobs_nmodel(nmodel_default_prime2[mask], n_obs[mask], ax, 'Nobs', 'fpix x Nmodel / weight', color='orange', add_scatter=False, label='Nbar x F^ x fpix', xlim=(-1, 8), ylim=(-1, 4))
ax.legend()

histogram of delta

In [ ]:
xdelta = np.linspace(-1, 10, 100)
_, bins,_= plt.hist(delta_fp[mask], range=(-1, 10), bins=40, alpha=0.5, color='b')
plt.hist(delta_fp[mask_t], bins=bins, alpha=0.6, label='0.5<Wsys<2.0', color='orange')
plt.plot(xdelta-1, 4.5e4*lognorm.pdf(xdelta, 0.7), label='lognorm')
plt.yscale('log')
plt.legend()
plt.xlabel(r'$\delta$')

In [ ]:
mask_t.sum() / mask.sum()

In [ ]:
from LSSutils.lab import AnaFast, MeanDensity

In [ ]:
# compute mean density vs ebv
frac = np.ones(mask.size)

nbar = MeanDensity(delta_i+1, frac, mask, ebv)
nbar.run()
nbarg = MeanDensity(delta_fg+1, frac, mask, ebv)
nbarg.run()
nbarp = MeanDensity(delta_fp+1, frac, mask, ebv)
nbarp.run()

In [ ]:
from LSSutils.lab import overdensity

In [ ]:
delta_nn = overdensity(n_obs_raw, frac_raw, mask_t, selection_fn=wsys_default)

In [ ]:
delta_nn_p = overdensity(n_obs_raw, frac_raw, mask_t, selection_fn=w_syspoi)

In [ ]:
clnnp = af(delta_nn_p, frac_raw, mask_t)

In [ ]:
# compute angular C_ell
af = AnaFast()
cl = af(delta_i, frac_raw, mask_t)
clg = af(delta_fg, frac_raw, mask_t)
clp = af(delta_fp, frac_raw, mask_t)

In [ ]:
clnn = af(delta_nn, frac_raw, mask_t)

In [ ]:
cl_raw = af(delta_raw_default, frac_raw, mask_t)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

# mk=['o', 's', 'x']

# for i, nbar_i in enumerate([nbar, nbarg, nbarp]):
    
#     ax[0].plot(nbar_i.output['bin_avg'], 
#                nbar_i.output['nnbar'], 
#                marker=mk[i])
    
# ax[0].set(xlabel='EBV', ylabel='N/Nbar', xscale='log')

names = ['No W',  'Pois.'] # 'Gauss.',

    
ax[1].loglog(cl_raw['l'], cl_raw['cl'], 'k-')    
ax[1].loglog(cl['l'], cl['cl'], label='No W')
ax[1].loglog(clp['l'], clp['cl'], label='w Poiss. Nmodel')
ax[1].loglog(clp['l'], clnnp['cl'], 'r--', label='w Poiss. Nmodel')
ax[1].loglog(clg['l'], clg['cl'], label='w F^')
ax[1].loglog(clnn['l'], clnn['cl'], 'k--', label='w F^')
    
ax[1].set_ylim(1.0e-7, 1.0e-2)    
ax[1].legend()
ax[1].set(ylabel=r'$C_{\ell}$', xlabel=r'$\ell$')

In [ ]:
from scipy.stats import binned_statistic

In [ ]:
def binit(x, values, frac, bins):
    n_, be_, _ = binned_statistic(x, values, statistic='sum', bins=bins)
    f_, be_, _ = binned_statistic(x, frac, statistic='sum', bins=bins)
    bc = 0.5*(be_[1:]+be_[:-1])
    return bc, (n_/f_) / (n_.sum()/f_.sum())    

In [ ]:
bins =  np.logspace(np.log10(ebv.min()), np.log10(ebv.max()), num=8)

In [ ]:
true = binit(ebv, n_true, frac, bins)
model1 = binit(ebv, y_poi['pred'], frac, bins)
model2 = binit(ebv, y_gau['pred'], frac, bins)
#model3 = binit(ebv, y_poiw['pred'], frac, bins)

In [ ]:
plt.plot(*true, marker='o', mfc='w')
plt.plot(*model1, marker='o')
plt.plot(*model2, marker='s')
# plt.plot(*model3, marker='s')
plt.legend(['Observed', 'PNLL', 'MSE'], loc=3)#, 'NPLL'])
plt.xlabel('EBV')
plt.xscale('log')
plt.ylabel(r'$\overline{\rm Ngal}$')

In [ ]:
plt.scatter(ebv, n_true, alpha=0.5, label='Observed Ngal')
plt.scatter(ebv, y_poi['pred'], alpha=0.2, label=r'$\lambda$')
plt.legend()
plt.xlabel('EBV')
plt.ylabel('Ngal')

In [ ]:
((n_true-y_gau['pred'])**2).mean()

In [ ]:
y_gau.keys()

In [ ]:
y_gau['metrics']['partition_0'].keys()

In [ ]:
plt.plot(y_poi['metrics']['partition_0']['val_losses'], 'r--',
        y_poi['metrics']['partition_0']['train_losses'], 'k-')
plt.legend(['Validation', 'Training'])

plt.axhline(y_poi['metrics']['partition_0']['base_train_loss'], ls=':', c='k')
plt.axhline(y_poi['metrics']['partition_0']['base_val_loss'], ls=':', c='r')
plt.ylim(0.965, 0.997)
plt.ylabel('PNLL')
plt.xlabel('EPOCH')

In [ ]:
plt.plot(y_gau['metrics']['partition_0']['val_losses'], 'r--',
         y_gau['metrics']['partition_0']['train_losses'], 'k-')
plt.legend(['Validation', 'Training'])

plt.axhline(y_gau['metrics']['partition_0']['base_train_loss'], ls=':', c='k')
plt.axhline(y_gau['metrics']['partition_0']['base_val_loss'], ls=':', c='r')
plt.ylim(0.9, 0.955)
plt.ylabel('MSE')
plt.xlabel('EPOCH')

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
x = torch.randn((100, 1))

In [ ]:
plt.scatter(x, F.softplus(x))
plt.scatter(x, np.log(1+np.exp(x)), 
            color='orange', marker='x', label='Softplus')
plt.scatter(x, F.relu(x), color='r', marker='.', label='RELU')
plt.legend()